In [1]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.8 MB/s eta 0:00:00


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
import keras_tuner as kt

In [3]:
#  Import and read the charity_data.csv
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
application_df.columns

Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = (list(application_counts[application_counts<500].index))
application_types_to_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [9]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

In [13]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [14]:
application_df.head(6)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1


In [15]:
# Convert categorical data to numeric with `pd.get_dummies`
dummified_df = pd.get_dummies(application_df, dtype=int)
dummified_df.head(6)

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
5,1,5000,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
# Split our preprocessed data into our features and target arrays
X = dummified_df.drop('IS_SUCCESSFUL', axis=1)
y = dummified_df['IS_SUCCESSFUL']

In [17]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

In [18]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Attempt 1

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
model_1 = tf.keras.models.Sequential()

# First hidden layer
model_1.add(Dense(units =70, activation = 'relu', input_dim = X_train.shape[1]))

# Second hidden layer
model_1.add(Dense(units = 35, activation = 'relu'))

# Output layer
model_1.add(Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 70)                3080      
                                                                 
 dense_1 (Dense)             (None, 35)                2485      
                                                                 
 dense_2 (Dense)             (None, 1)                 36        
                                                                 
Total params: 5601 (21.88 KB)
Trainable params: 5601 (21.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
# Compile the model
model_1.compile(loss = "binary_crossentropy",
                  optimizer = "adam",
                  metrics = ["accuracy"])

In [21]:
# Train the model
model_1.fit(X_train_scaled, y_train, epochs = 20, shuffle=True, verbose = 2)

Epoch 1/20
804/804 - 4s - loss: 0.5741 - accuracy: 0.7198 - 4s/epoch - 5ms/step
Epoch 2/20
804/804 - 3s - loss: 0.5550 - accuracy: 0.7297 - 3s/epoch - 3ms/step
Epoch 3/20
804/804 - 3s - loss: 0.5514 - accuracy: 0.7311 - 3s/epoch - 4ms/step
Epoch 4/20
804/804 - 5s - loss: 0.5503 - accuracy: 0.7326 - 5s/epoch - 6ms/step
Epoch 5/20
804/804 - 3s - loss: 0.5490 - accuracy: 0.7325 - 3s/epoch - 3ms/step
Epoch 6/20
804/804 - 2s - loss: 0.5475 - accuracy: 0.7328 - 2s/epoch - 3ms/step
Epoch 7/20
804/804 - 2s - loss: 0.5467 - accuracy: 0.7338 - 2s/epoch - 3ms/step
Epoch 8/20
804/804 - 1s - loss: 0.5463 - accuracy: 0.7339 - 1s/epoch - 2ms/step
Epoch 9/20
804/804 - 2s - loss: 0.5453 - accuracy: 0.7338 - 2s/epoch - 2ms/step
Epoch 10/20
804/804 - 2s - loss: 0.5450 - accuracy: 0.7346 - 2s/epoch - 2ms/step
Epoch 11/20
804/804 - 2s - loss: 0.5441 - accuracy: 0.7349 - 2s/epoch - 2ms/step
Epoch 12/20
804/804 - 1s - loss: 0.5436 - accuracy: 0.7356 - 1s/epoch - 2ms/step
Epoch 13/20
804/804 - 1s - loss: 0.54

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = model_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5596 - accuracy: 0.7272 - 735ms/epoch - 3ms/step
Loss: 0.5596309900283813, Accuracy: 0.7272303104400635


In [25]:
model1_df = pd.DataFrame({'Loss': [model_loss], 'Accuracy': [model_accuracy]})

# Attempt 2

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units =64, activation = 'relu', input_dim = X_train.shape[1]))

# Second hidden layer
nn.add(Dense(units = 32, activation = 'relu'))

# Output layer
nn.add(Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                2816      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4929 (19.25 KB)
Trainable params: 4929 (19.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss = "binary_crossentropy",
                  optimizer = "adam",
                  metrics = ["accuracy"])

In [29]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs = 50, shuffle=True, verbose = 2)

Epoch 1/50
804/804 - 2s - loss: 0.5727 - accuracy: 0.7186 - 2s/epoch - 3ms/step
Epoch 2/50
804/804 - 1s - loss: 0.5549 - accuracy: 0.7298 - 1s/epoch - 2ms/step
Epoch 3/50
804/804 - 2s - loss: 0.5517 - accuracy: 0.7305 - 2s/epoch - 2ms/step
Epoch 4/50
804/804 - 2s - loss: 0.5496 - accuracy: 0.7309 - 2s/epoch - 3ms/step
Epoch 5/50
804/804 - 2s - loss: 0.5486 - accuracy: 0.7334 - 2s/epoch - 2ms/step
Epoch 6/50
804/804 - 1s - loss: 0.5472 - accuracy: 0.7329 - 1s/epoch - 2ms/step
Epoch 7/50
804/804 - 1s - loss: 0.5462 - accuracy: 0.7345 - 1s/epoch - 2ms/step
Epoch 8/50
804/804 - 1s - loss: 0.5455 - accuracy: 0.7338 - 1s/epoch - 2ms/step
Epoch 9/50
804/804 - 1s - loss: 0.5451 - accuracy: 0.7345 - 1s/epoch - 2ms/step
Epoch 10/50
804/804 - 1s - loss: 0.5446 - accuracy: 0.7349 - 1s/epoch - 2ms/step
Epoch 11/50
804/804 - 1s - loss: 0.5441 - accuracy: 0.7347 - 1s/epoch - 2ms/step
Epoch 12/50
804/804 - 1s - loss: 0.5438 - accuracy: 0.7367 - 1s/epoch - 2ms/step
Epoch 13/50
804/804 - 2s - loss: 0.54

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5672 - accuracy: 0.7236 - 505ms/epoch - 2ms/step
Loss: 0.5671907067298889, Accuracy: 0.7236151695251465


In [31]:
model2_df = pd.DataFrame({'Loss': [model_loss], 'Accuracy': [model_accuracy]})

# Attempt 3

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units =300, activation = 'sigmoid', input_dim = X_train.shape[1]))

# Second hidden layer
nn.add(Dense(units = 150, activation = 'sigmoid'))

# Output layer
nn.add(Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 300)               13200     
                                                                 
 dense_7 (Dense)             (None, 150)               45150     
                                                                 
 dense_8 (Dense)             (None, 1)                 151       
                                                                 
Total params: 58501 (228.52 KB)
Trainable params: 58501 (228.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
# Compile the model
nn.compile(loss = "binary_crossentropy",
                  optimizer = "adam",
                  metrics = ["accuracy"])

In [35]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs = 100, shuffle=True, verbose = 2)

Epoch 1/100
804/804 - 3s - loss: 0.5913 - accuracy: 0.7069 - 3s/epoch - 4ms/step
Epoch 2/100
804/804 - 3s - loss: 0.5718 - accuracy: 0.7235 - 3s/epoch - 4ms/step
Epoch 3/100
804/804 - 2s - loss: 0.5659 - accuracy: 0.7252 - 2s/epoch - 3ms/step
Epoch 4/100
804/804 - 2s - loss: 0.5627 - accuracy: 0.7269 - 2s/epoch - 3ms/step
Epoch 5/100
804/804 - 2s - loss: 0.5595 - accuracy: 0.7280 - 2s/epoch - 3ms/step
Epoch 6/100
804/804 - 2s - loss: 0.5572 - accuracy: 0.7288 - 2s/epoch - 3ms/step
Epoch 7/100
804/804 - 2s - loss: 0.5554 - accuracy: 0.7302 - 2s/epoch - 3ms/step
Epoch 8/100
804/804 - 3s - loss: 0.5549 - accuracy: 0.7298 - 3s/epoch - 4ms/step
Epoch 9/100
804/804 - 2s - loss: 0.5532 - accuracy: 0.7308 - 2s/epoch - 3ms/step
Epoch 10/100
804/804 - 2s - loss: 0.5527 - accuracy: 0.7306 - 2s/epoch - 3ms/step
Epoch 11/100
804/804 - 2s - loss: 0.5527 - accuracy: 0.7304 - 2s/epoch - 3ms/step
Epoch 12/100
804/804 - 2s - loss: 0.5520 - accuracy: 0.7310 - 2s/epoch - 3ms/step
Epoch 13/100
804/804 - 2s

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5826 - accuracy: 0.7231 - 534ms/epoch - 2ms/step
Loss: 0.5825752019882202, Accuracy: 0.7231487035751343


In [40]:
model3_df = pd.DataFrame({'Loss': [model_loss], 'Accuracy': [model_accuracy]})

In [50]:
analysis = pd.concat([model1_df, model2_df, model3_df], ignore_index=True)
analysis_summary = analysis.rename(index={0: 'attempt_1', 1: 'attempt_2', 2: 'attempt_3'})


In [49]:
analysis_summary.round(2)

,Loss,Accuracy
attempt_1,0.56,0.73
attempt_2,0.58,0.72
attempt_3,0.58,0.72


In [51]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.keras")